In [1]:
!pip install stemming


In [2]:
import string
import re
from nltk.corpus import stopwords
from stemming.porter2 import stem
import networkx
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bhavika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
fhr = open('amazon-meta.txt', 'r', encoding='utf-8', errors='ignore')
amazonProducts = {}

In [4]:
(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)

In [5]:
for line in fhr:
    line = line.strip()
    # a product block started
    if(line.startswith("Id")):
        Id = line[3:].strip()
        if(Id == "200000"):
            break;
    elif(line.startswith("ASIN")):
        ASIN = line[5:].strip()
    elif(line.startswith("title")):
        Title = line[6:].strip()
        Title = ' '.join(Title.split())
        
    elif(line.startswith("group")):
        Group = line[6:].strip()
    elif(line.startswith("salesrank")):
        SalesRank = line[10:].strip()
    elif(line.startswith("similar")):
        ls = line.split()
        Copurchased = ' '.join([c for c in ls[2:]])
    elif(line.startswith("categories")):
        ls = line.split()
        Categories = ' '.join((fhr.readline()).lower() for i in range(int(ls[1].strip())))
        Categories = re.compile('[%s]' % re.escape(string.digits + string.punctuation)).sub(' ', Categories)
        Categories = ' '.join(set(Categories.split()) - set(stopwords.words("english")))
        Categories = ' '.join(stem(word) for word in Categories.split())
    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
    # product block end
    # write out fields to amazonProducts dictionary
    elif(line==""):
        try:
            MetaData={}
            if(ASIN != ""):
                amazonProducts[ASIN] = MetaData
            MetaData['Id'] = Id
            MetaData['Title'] = Title
            MetaData['Categories'] = ' '.join(set(Categories.split()))
            MetaData['Group'] = Group
            MetaData['Copurchased'] = Copurchased
            MetaData['SalesRank'] = int(SalesRank)
            MetaData['TotalReviews'] = int(TotalReviews)
            MetaData['AvgRating'] = float(AvgRating)
            MetaData['DegreeCentrality'] = DegreeCentrality
            MetaData['ClusteringCoeff'] = ClusteringCoeff
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = \
            ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)
fhr.close()

In [9]:
cm =0
cd = 0
cv = 0
for asin,metadata in amazonProducts.items():
    
    if(metadata['Group']=='Music'):
        cm+=1
    elif(metadata['Group']=='Book'):
        cv+=1
    elif(metadata['Group']=='DVD'):
        cd+=1
    else:
        continue
print(cm)
print(cv)
print(cd)

37612
143309
7135


In [10]:
# create books-specific dictionary exclusively for books
amazonBooks = {}
for asin,metadata in amazonProducts.items():
    if (metadata['Group']=='Book'):
        amazonBooks[asin] = amazonProducts[asin]

In [11]:
len(amazonBooks)

143309

In [12]:
# remove any copurchased items from copurchase list if we don't have metadata associated with it
for asin, metadata in amazonBooks.items():
    amazonBooks[asin]['Copurchased'] = \
        ' '.join([cp for cp in metadata['Copurchased'].split() \
                  if cp in amazonBooks.keys()])


In [13]:
# Create a product copurchase graph for analysis where the graph nodes for product ASINs
# and graph edge exists if two products were copurchased,
# with edge weight being a measure of category similarity between ASINs
###amazonBooks['0827229534']['Copurchased'].split()
    
copurchaseGraphBooks = networkx.Graph()
for asin,metadata in amazonBooks.items():
    copurchaseGraphBooks.add_node(asin)
    for a in metadata['Copurchased'].split():
        copurchaseGraphBooks.add_node(a.strip())
        similarity = 0
        n1 = set((amazonBooks[asin]['Categories']).split())
        n2 = set((amazonBooks[a]['Categories']).split())
        n1In2 = n1 & n2     # intersection (Number of words that are common between Categories of connected Nodes)
        n1Un2 = n1 | n2     # union (Total number of words in both Categories of connected Nodes)
        if(len(n1Un2)) > 0:
            similarity = round(len(n1In2)/len(n1Un2), 2)
        copurchaseGraphBooks.add_edge(asin, a.strip(), weight=similarity)

In [15]:
# Get Degree Centrality and clustering coefficients
# of each ASIN and add it to amazonBooks metadata
dc = networkx.degree(copurchaseGraphBooks)
for asin in networkx.nodes(copurchaseGraphBooks):
    metadata = amazonBooks[asin]
    metadata['DegreeCentrality'] = int(dc[asin])
    ego = networkx.ego_graph(copurchaseGraphBooks, asin, radius=1)
    metadata['ClusteringCoeff'] = round(networkx.average_clustering(ego), 2)
    amazonBooks[asin] = metadata
    
counter = 0
for f,t,e in copurchaseGraphBooks.edges(data=True):
    if counter == 150:
        break
    print(e['weight'])
    counter += 1

0.7
0.8
0.7
0.7
0.75
1.0
0.9
0.78
0.39
0.5
0.33
0.83
0.8
0.58
0.1
0.8
0.7
0.35
0.5
0.31
0.19
0.1
0.39
0.48
0.48
0.39
0.25
0.9
1.0
0.57
0.73
1.0
0.55
0.35
0.56
0.15
0.12
0.2
0.48
0.47
0.66
0.14
0.23
0.25
0.18
0.2
0.1
0.18
0.19
0.33
0.33
0.25
0.27
0.31
0.23
0.31
0.33
0.25
0.0
0.32
0.38
0.53
0.42
0.64
0.41
0.25
0.29
0.38
0.36
0.71
0.71
1.0
0.25
0.25
1.0
0.89
1.0
0.53
0.41
0.67
0.8
0.44
0.73
0.89
0.89
0.44
0.57
0.8
0.6
0.78
0.53
0.53
0.43
0.48
1.0
0.48
0.5
0.4
0.48
0.4
0.3
0.22
0.48
0.25
0.39
0.3
0.38
0.24
0.5
0.24
0.41
0.38
0.37
0.26
0.26
0.45
0.24
0.38
0.22
0.17
0.65
0.47
0.31
0.1
0.63
0.59
0.45
0.59
0.55
0.27
0.61
0.46
0.39
0.39
0.48
0.7
0.8
0.5
0.59
0.59
0.36
0.28
0.63
0.34
0.39
0.33
0.3
0.37
0.38
0.12


In [16]:
# Write amazonBooks data to file
fhw = open('amazon-books.txt', 'w', encoding='utf-8', errors='ignore')
fhw.write("Id\t" + "ASIN\t" + "Title\t" + "Categories\t" + "Group\t" + "Copurchased\t" + "SalesRank\t" + "TotalReviews\t" + "AvgRating\t" + "DegreeCentrality\t" + "ClusteringCoeff\n")
for asin, metadata in amazonBooks.items():
    fhw.write(metadata['Id'] + "\t" + \
              asin + "\t" + \
              metadata['Title'] + "\t" + \
              metadata['Categories'] + "\t" + \
              metadata['Group'] + "\t" + \
              metadata['Copurchased'] + "\t" + \
              str(metadata['SalesRank']) + "\t" + \
              str(metadata['TotalReviews']) + "\t" + \
              str(metadata['AvgRating']) + "\t" + \
              str(metadata['DegreeCentrality']) + "\t" + \
              str(metadata['ClusteringCoeff']) + "\n")
fhw.close()

In [17]:
# Write copurchaseGraph data to file
fhw = open('amazon-books-copurchase.edgelist', 'wb')
networkx.write_weighted_edgelist(copurchaseGraphBooks, fhw)
fhw.close()

In [18]:
amazonMusic = {}
for asin,metadata in amazonProducts.items():
    if (metadata['Group']=='Music'):
        amazonMusic[asin] = amazonProducts[asin]

In [19]:
len(amazonMusic)

37612

In [20]:
# remove any copurchased items from copurchase list if we don't have metadata associated with it
for asin, metadata in amazonMusic.items():
    amazonMusic[asin]['Copurchased'] = \
        ' '.join([cp for cp in metadata['Copurchased'].split() \
                  if cp in amazonMusic.keys()])


In [21]:
#copurchase graph for music

copurchaseGraphMusic = networkx.Graph()
for asin,metadata in amazonMusic.items():
    copurchaseGraphMusic.add_node(asin)
    for a in metadata['Copurchased'].split():
        copurchaseGraphMusic.add_node(a.strip())
        similarity = 0
        n1 = set((amazonMusic[asin]['Categories']).split())
        n2 = set((amazonMusic[a]['Categories']).split())
        n1In2 = n1 & n2     # intersection (Number of words that are common between Categories of connected Nodes)
        n1Un2 = n1 | n2     # union (Total number of words in both Categories of connected Nodes)
        if(len(n1Un2)) > 0:
            similarity = round(len(n1In2)/len(n1Un2), 2)
        copurchaseGraphMusic.add_edge(asin, a.strip(), weight=similarity)

In [22]:
#degree centrality and coeff
dc = networkx.degree(copurchaseGraphMusic)
for asin in networkx.nodes(copurchaseGraphMusic):
    metadata = amazonMusic[asin]
    metadata['DegreeCentrality'] = int(dc[asin])
    ego = networkx.ego_graph(copurchaseGraphMusic, asin, radius=1)
    metadata['ClusteringCoeff'] = round(networkx.average_clustering(ego), 2)
    amazonMusic[asin] = metadata
    
counter = 0
for f,t,e in copurchaseGraphMusic.edges(data=True):
    if counter == 150:
        break
    print(e['weight'])
    counter += 1

0.28
0.31
0.86
0.38
0.86
1.0
0.36
0.67
0.19
0.33
1.0
1.0
0.73
1.0
0.45
0.64
0.56
0.73
0.31
0.4
0.0
0.0
1.0
1.0
0.8
0.33
0.57
0.36
0.45
1.0
1.0
1.0
0.67
1.0
1.0
1.0
1.0
0.24
0.88
1.0
0.88
1.0
1.0
0.28
0.88
0.88
0.88
1.0
0.74
0.88
0.25
0.25
1.0
0.41
0.4
0.4
0.27
0.29
0.33
0.22
0.14
0.15
0.08
0.75
0.17
0.92
1.0
0.92
0.92
0.48
0.34
0.4
1.0
1.0
0.4
1.0
0.4
0.47
0.7
0.27
0.7
0.4
0.7
0.22
0.7
0.46
0.7
0.73
1.0
0.73
1.0
1.0
1.0
1.0
0.5
0.29
0.28
0.77
1.0
0.46
0.25
0.77
0.46
1.0
1.0
1.0
0.21
0.62
0.33
0.17
0.2
0.4
0.62
0.71
0.32
0.36
0.5
0.93
0.76
0.56
0.56
0.6
0.5
0.94
1.0
0.37
0.48
0.39
0.62
0.61
0.19
0.94
0.35
0.87
0.91
0.89
0.89
0.38
0.95
0.45
0.43
0.43
0.25
0.38
0.31
0.0
0.73
0.32
0.84
0.68


In [23]:
fhw = open('amazon-music.txt', 'w', encoding='utf-8', errors='ignore')
fhw.write("Id\t" + "ASIN\t" + "Title\t" + "Categories\t" + "Group\t" + "Copurchased\t" + "SalesRank\t" + "TotalReviews\t" + "AvgRating\t" + "DegreeCentrality\t" + "ClusteringCoeff\n")
for asin, metadata in amazonMusic.items():
    fhw.write(metadata['Id'] + "\t" + \
              asin + "\t" + \
              metadata['Title'] + "\t" + \
              metadata['Categories'] + "\t" + \
              metadata['Group'] + "\t" + \
              metadata['Copurchased'] + "\t" + \
              str(metadata['SalesRank']) + "\t" + \
              str(metadata['TotalReviews']) + "\t" + \
              str(metadata['AvgRating']) + "\t" + \
              str(metadata['DegreeCentrality']) + "\t" + \
              str(metadata['ClusteringCoeff']) + "\n")
fhw.close()

In [24]:
fhw = open('amazon-music-copurchase.edgelist', 'wb')
networkx.write_weighted_edgelist(copurchaseGraphMusic, fhw)
fhw.close()